In [43]:
import numpy as np
import pandas as pd
import functions as func 
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
import re
from scipy import stats
from scipy.special import boxcox, inv_boxcox



In [44]:
data = pd.read_csv("D:/downloads/project/Scraper/venv/all_internships_210123_183812.csv",encoding='latin1')
data.head()

,category,company,location,start_date,duration,stipend,apply_by,applicants,skills_required,perks,openings,link
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://internshala.com/job/detail/inside-sale...
1,Cloud Development (Backend),Techwondoe India Private Limited,"Work From Home,",Immediately,3 Months,"10,000-25,000 /month",5 Feb' 23,861.0,"Amazon Web Server (AWS),Node.js,REST API,Tailw...","Certificate,Letter of recommendation,5 days a ...",2.0,https://internshala.com/internship/detail/clou...
2,Fundraising,Tare Zameen Foundation,"Work From Home,",Immediately,1 Month,"1,000 /month + Incentives",3 Feb' 23,186.0,NaN,"Certificate,Letter of recommendation,Flexible ...",200.0,https://internshala.com/internship/detail/fund...
3,Brand Management &amp; Marketing,Corizo,"Work From Home,",Immediately,1 Month,"1,000 /month + Incentives",2 Feb' 23,405.0,"Email Marketing,Instagram Marketing,Social Med...","Certificate,Letter of recommendation,Job offer,",20.0,https://internshala.com/internship/detail/bran...
4,Android Teaching Assistant (Virtual),Internshala Trainings,"Work From Home,",Immediately,6 Months,"5,000 /month + Incentives",27 Jan' 23,225.0,"Android,Kotlin,","Certificate,Flexible work hours,",2.0,https://internshala.com/internship/detail/andr...


In [45]:
emptyRowIndex = data[data['company'].isna()].index

In [46]:
data = data.drop(index=emptyRowIndex)

In [47]:
print(data[data['stipend'].str.match(r'^[a-zA-Z]+',na=False)]['stipend'].unique())

['Performance Based' 'Unpaid' 'Not provided']


In [48]:
unpaid_data = data[(data['stipend']=='Performance Based') | (data['stipend']=='Unpaid')]
# Dropping these values from main data
data.drop(unpaid_data.index, inplace=True)

In [49]:
data.drop(data[data['stipend']=='Not provided'].index, inplace=True)
data.drop(data[data['stipend'].isna()].index, inplace=True)

In [50]:
_ = data['stipend'].apply(lambda x: str(x).split(' ')[0].replace(',',"").split("-"))
lower = (_.apply(lambda x:x[0])).astype('float32')
upper = (_.apply(lambda x:x[-1])).astype('float32')
data['stipend'] = (lower+upper)/2

In [51]:
print(data['duration'].isna().sum())

0


In [52]:
print(data['duration'].value_counts())

duration
3 Months        3486
6 Months        2987
2 Months        1214
1 Month          604
4 Months         479
5 Months          79
1 Week            34
2 Weeks           34
6 Weeks           26
4 Weeks           12
36 Months          9
24 Months          7
3 Weeks            7
12 Months          6
Not Provided       2
12 months          1
Name: count, dtype: int64


In [53]:
data.drop(data[data['duration']=='Not Provided'].index, inplace=True)

In [54]:
week_or_day_multiplier = (data['duration'].apply(lambda x: 4 if "Week" in str(x) else 1))
# Taking out the only integer value of duration column and dividing it with week_or_day_multiplier so that all the values are in months only.
data['duration'] = (data['duration'].apply(lambda x: int(str(x).split(' ')[0]))/week_or_day_multiplier)

In [55]:
print(data['duration'].value_counts())

duration
3.00     3486
6.00     2987
2.00     1214
1.00      616
4.00      479
5.00       79
0.25       34
0.50       34
1.50       26
36.00       9
24.00       7
12.00       7
0.75        7
Name: count, dtype: int64


In [56]:
data = data.groupby('company').filter(lambda x: len(x) > 1)

In [57]:
data.shape

(4454, 12)

In [58]:
print(data['category'].isna().sum())

0


In [59]:
print(data['category'].nunique())

1276


In [60]:
data['category'] = (data['category'].apply(lambda x: str(x).split('(')[0].split('-')[0].split('&amp;')[0].strip()))
data['category'] = (data['category'].apply(lambda x: 'Teaching' if 'teaching' in str(x).lower() else x))
data['category'] = (data['category'].apply(lambda x: '2D/3D' if '2D' in str(x) else x))
data['category'] = (data['category'].apply(lambda x: '2D/3D' if '3D' in str(x) else x))
data['category'] = (data['category'].apply(lambda x: 'Content Writing/Creation Related' if 'content' in str(x).lower() else x))
data['category'] = (data['category'].apply(lambda x: 'Graphic Desing Realted' if 'graphic' in str(x).lower() else x))
data['category'] = (data['category'].apply(lambda x: 'Video Editing/Making/Animation Related' if 'video' in str(x).lower() else x))
data['category'] = (data['category'].apply(lambda x: 'Database Operations' if 'database' in str(x).lower() else x))
data['category'] = (data['category'].apply(lambda x: 'Data_Entry Related' if 'data entry' in str(x).lower() else x))
data['category'] = (data['category'].apply(lambda x: 'Data_Entry Related' if 'data typing' in str(x).lower() else x))
data['category'] = (data['category'].apply(lambda x: 'Data_Annotation' if 'data annotation' in str(x).lower() else x))
data['category'] = (data['category'].apply(lambda x: 'Data_Analysis/Data_Science' if 'data' in str(x).lower() else x))
data['category'] = (data['category'].apply(lambda x: 'Research' if 'research' in str(x).lower() else x))
data['category'] = (data['category'].apply(lambda x: 'Web Development/Design' if 'web ' in str(x).lower() else x))
data['category'] = (data['category'].apply(lambda x: 'UI/UX' if 'ui' in str(x).lower() else x))
data['category'] = (data['category'].apply(lambda x: 'UI/UX' if 'ux' in str(x).lower() else x))
data['category'] = (data['category'].apply(lambda x: 'B2B Related' if 'b2b' in str(x).lower() else x))
data['category'] = (data['category'].apply(lambda x: 'Google AdWords' if 'google ad' in str(x).lower() else x))
data['category'] = (data['category'].apply(lambda x: 'Social Media Related' if 'social media' in str(x).lower() else x))
data['category'] = (data['category'].apply(lambda x: 'Journalism' if 'journalism' in str(x).lower() else x))
data['category'] = (data['category'].apply(lambda x: 'Subject Matter Expert' if 'subject matter' in str(x).lower() else x))
data['category'] = (data['category'].apply(lambda x: 'Teaching' if 'tutor' in str(x).lower() else x))
data['category'] = (data['category'].apply(lambda x: x.replace('Analytics','Analysis').replace('Analyst','Analysis')))
data['category'] = (data['category'].apply(lambda x: 'Business Analysis' if 'business analysis' in str(x).lower() else x))
data['category'] = (data['category'].apply(lambda x: 'Operations' if 'operations ' in str(x).lower() else x))
data['category'] = (data['category'].apply(lambda x: 'Finance' if 'financial' in str(x).lower() else x))
data['category'] = (data['category'].apply(lambda x: 'Finance' if 'finance' in str(x).lower() else x))
data['category'] = (data['category'].apply(lambda x: 'Architecture' if 'architectur' in str(x).lower() else x))
data['category'] = (data['category'].apply(lambda x: 'Search Engine Optimization' if 'seo' in str(x).lower() else x))
data['category'] = (data['category'].apply(lambda x: '.js Development' if '.js' in str(x).lower() else x))
data['category'] = (data['category'].apply(lambda x: '.js Development' if 'react' in str(x).lower() else x))
data['category'] = (data['category'].apply(lambda x: '.js Development' if 'angular' in str(x).lower() else x))
data['category'] = (data['category'].apply(lambda x: 'Human Resources' if 'hr ' in str(x).lower() else x))
data['category'] = (data['category'].apply(lambda x: 'Human Resources' if '/hr' in str(x).lower() else x))
data['category'] = (data['category'].apply(lambda x: 'Human Resources' if 'HR' in str(x) else x))
data['category'] = (data['category'].apply(lambda x: 'Charity' if 'charity' in str(x).lower() else x))
data['category'] = (data['category'].apply(lambda x: 'Anchoring' if 'anchor' in str(x).lower() else x))
data['category'] = (data['category'].apply(lambda x: 'CAD' if 'cad ' in str(x).lower() else x))
data['category'] = (data['category'].apply(lambda x: 'Chartered Accountancy' if 'ca ' in str(x).lower() else x))
data['category'] = (data['category'].apply(lambda x: 'Chartered Accountancy' if 'ca/' in str(x).lower() else x))
data['category'] = (data['category'].apply(lambda x: 'Chartered Accountancy' if 'CA' in str(x) else x))
data['category'] = (data['category'].apply(lambda x: 'Academics Related' if 'academic' in str(x).lower() else x))
data['category'] = (data['category'].apply(lambda x: 'Website Development Related' if bool(re.search('(?i)web(site|page|flow|/E|[3])? ?($|\s)',str(x).lower())) else x))

In [61]:
unique_categories = (data['category'].unique().tolist())

In [62]:
# Making a different array of skills with elements as list
skills_array = data['skills_required'].apply(lambda x: str(x).split(',')[:-1])
# Creating a 1d list
skills_list = [item for sublist in skills_array for item in sublist]

#Counting the occurences of each skill
from collections import Counter
skills_list = dict(Counter(skills_list))

# Sorting in ascending order for later use
sorted_skills_list = sorted(skills_list.keys())
# Sorting in descending order for plotting purpose
skills_list_toPlot = dict(sorted(skills_list.items(),key=lambda x:x[1], reverse=True))
reverse_skills_dict = dict((sorted(skills_list.items(),key=lambda x:x[1],reverse=False)))

print("All Skills with their occurences: ",skills_list_toPlot)
print("Total unique skills: ",len(skills_list_toPlot))

All Skills with their occurences:  {'English Proficiency (Spoken)': 1344, 'English Proficiency (Written)': 1343, 'Social Media Marketing': 712, 'MS-Excel': 674, 'Digital Marketing': 671, 'Creative Writing': 457, 'MS-Office': 446, 'Adobe Photoshop': 443, 'Search Engine Optimization (SEO)': 424, 'Email Marketing': 386, 'Adobe Illustrator': 346, 'Instagram Marketing': 293, 'Video Editing': 284, 'Facebook Marketing': 247, 'Adobe After Effects': 220, 'Search Engine Marketing (SEM)': 220, 'MS-Word': 219, 'Adobe Premiere Pro': 207, 'HTML': 190, 'Content Writing': 177, 'JavaScript': 172, 'Hindi Proficiency (Spoken)': 166, 'Sales and Marketing': 161, 'CSS': 150, 'Video Making': 145, 'MS-PowerPoint': 139, 'CorelDRAW': 139, 'Blogging': 136, 'UI & UX Design': 127, 'Python': 115, 'Content Marketing': 112, 'Adobe Indesign': 106, 'Canva': 104, 'PHP': 103, 'Adobe Creative Suite': 102, 'ReactJS': 97, 'WordPress': 97, 'Java': 93, 'Node.js': 92, 'Accounting': 91, 'Sales': 91, 'MySQL': 89, 'Adobe Photosho

In [63]:
def leastOccuringSkill(skill_array,no_skills_grouped): # no_skills_grouped is the number of occurences of a skill that can be grouped with no effect on stipend
    skills_occurances = []
    for skill in skill_array:
        skills_occurances.append(reverse_skills_dict.get(skill))
    if(len(skills_occurances)==0):
        return np.nan
    min_value = min(skills_occurances)
    if min_value<=no_skills_grouped:
        return skill_array[skills_occurances.index(min_value)]
    else:
        return np.nan

def perform_anova_for_skills(skills_stipend_df,numberOfSkillsToBeGrouped):
    skills_stipend_df.loc[:,'least_skill'] = (skills_stipend_df['skills_required'].apply(lambda x: leastOccuringSkill(str(x).split(',')[:-1],numberOfSkillsToBeGrouped)))
    skills_stipend_df.dropna(subset=['least_skill'],inplace=True)
    skills_stipend_df = skills_stipend_df[['least_skill','stipend']]
    result=func.anova(skills_stipend_df,'least_skill','stipend')
    print("ANOVA result for grouping skills with ",numberOfSkillsToBeGrouped," occurences :",result)
    return result[1]

In [64]:
skills_stipend = data[['skills_required','stipend']]

grouping_skills_with_occurences=2
grouping_skills_with_occurences-=1 #To overcome '+1' in '*' line
while True:
    skills_stipend = data[['skills_required','stipend']] #To overcome slicing
    res = perform_anova_for_skills(skills_stipend,grouping_skills_with_occurences+1) # '*' line
    if res<0.05:
        break
    grouping_skills_with_occurences+=1

print("Skills with ",grouping_skills_with_occurences,' occurences can be grouped')

ANOVA result for grouping skills with  2  occurences : F_onewayResult(statistic=1.7745606239060656, pvalue=0.05635367457801497)
ANOVA result for grouping skills with  3  occurences : F_onewayResult(statistic=1.7431575719566557, pvalue=0.011972378623853312)
Skills with  2  occurences can be grouped


C:\Users\avikm\AppData\Local\Temp\ipykernel_17132\2819700676.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  skills_stipend_df.loc[:,'least_skill'] = (skills_stipend_df['skills_required'].apply(lambda x: leastOccuringSkill(str(x).split(',')[:-1],numberOfSkillsToBeGrouped)))
C:\Users\avikm\AppData\Local\Temp\ipykernel_17132\2819700676.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  skills_stipend_df.dropna(subset=['least_skill'],inplace=True)
C:\Users\avikm\AppData\Local\Temp\ipykernel_17132\2819700676.py:14: SettingWithCopyWarning: 
A value is trying to

In [65]:
skills_to_be_edited = reverse_skills_dict.copy()
key_list = list(skills_to_be_edited.keys())
for key in key_list:
    if skills_to_be_edited.get(key)<=grouping_skills_with_occurences:
        skills_to_be_edited.update({key:'Others'})
    else:
        skills_to_be_edited.update({key:key})

unique_final_skills = (skills_to_be_edited.values())
print("Unique Skills after merging not significant skills as 'Others': ",pd.Series(unique_final_skills).nunique())

# Creating a txt file of all skills to use later


def modify_list(skill_list):
    new_list = []
    for list_item in skill_list:
        new_list.append(skills_to_be_edited.get(list_item))
    return new_list

data_skills = pd.DataFrame(data['skills_required'].apply(lambda x: modify_list(str(x).split(',')[:-1])))


Unique Skills after merging not significant skills as 'Others':  248


In [66]:
# Making a different array of location with elements as list
data['location']=(data['location'].apply(lambda x:x.replace(", ",",")))
location_array = data['location'].apply(lambda x: str(x).split(',')[:-1])
# Creating a 1d list
location_list = [item for sublist in location_array for item in sublist]

#Counting the occurences of each skill
from collections import Counter
location_list = dict(Counter(location_list))

# Sorting in ascending order for later use
sorted_location_list = sorted(location_list.keys())
# Sorting in descending order for plotting purpose
locations_list_toPlot = dict(sorted(location_list.items(),key=lambda x:x[1], reverse=True))
reverse_location_dict = dict((sorted(location_list.items(),key=lambda x:x[1],reverse=False)))

print("All Location with their occurences: ",locations_list_toPlot)
print("Total unique locations: ",len(locations_list_toPlot))

All Location with their occurences:  {'Work From Home': 1822, 'Mumbai': 551, 'Delhi': 503, 'Bangalore': 336, 'Noida': 248, 'Gurgaon': 248, 'Pune': 197, 'Jaipur': 111, 'Ahmedabad': 94, 'Hyderabad': 91, 'Navi Mumbai': 77, 'Chennai': 72, 'Mohali': 60, 'Ghaziabad': 56, 'Chandigarh': 56, 'Indore': 55, 'Thane': 52, 'Kolkata': 46, 'Faridabad': 33, 'Lucknow': 22, 'Vadodara': 22, 'Kalyan': 20, 'Coimbatore': 19, 'Greater Noida': 17, 'Dombivli': 16, 'Nagpur': 15, 'Gandhinagar': 15, 'Kochi': 15, 'Surat': 14, 'Dehradun': 14, 'Udaipur': 13, 'Ambernath': 13, 'Raipur': 12, 'Bhopal': 12, 'Ulhasnagar': 12, 'Varanasi': 12, 'Ludhiana': 11, 'Panvel': 11, 'Pimpri-Chinchwad': 10, 'Rajkot': 10, 'Mira Bhayandar': 10, 'Kharar': 10, 'Nashik': 10, 'Vashi': 9, 'Bhubaneswar': 9, 'Panchkula': 8, 'Gautam Buddha Nagar': 8, 'Bhiwandi': 7, 'Visakhapatnam': 7, 'Amritsar': 7, 'Jodhpur': 6, 'Kota': 6, 'Haryana': 6, 'Ernakulam': 6, 'Badlapur': 5, 'Patna': 5, 'Cochin': 5, 'Mehsana': 5, 'Patiala': 5, 'Bhilai': 5, 'Durg': 5, '

In [67]:
# Directly 'in' cannot be used as 'in' does not prioritize the least occuring location. As the locations_to_remove list arranged in ascending order of occurances, the least occuring location will be returned
def leastOccuringLocation(location_array,no_locations_grouped): # no_locations_grouped is the number of occurences of a location that can be grouped with no effect on stipend
    locations_occurances = []
    for location in location_array:
        locations_occurances.append(reverse_location_dict.get(location))
    if(len(locations_occurances)==0):
        return np.nan
    min_value = min(locations_occurances)
    if min_value<=no_locations_grouped:
        return location_array[locations_occurances.index(min_value)]
    else:
        return np.nan

def perform_anova_for_locations(locations_stipend_df,numberOfLocationsToBeGrouped):
    locations_stipend_df.loc[:,'least_location'] = (locations_stipend_df['location'].apply(lambda x: leastOccuringLocation(str(x).split(',')[:-1],numberOfLocationsToBeGrouped)))
    locations_stipend_df.dropna(subset=['least_location'],inplace=True)
    locations_stipend_df = locations_stipend_df[['least_location','stipend']]
    result=func.anova(locations_stipend_df,'least_location','stipend')
    print("ANOVA result for grouping locations with ",numberOfLocationsToBeGrouped," occurences :",result)
    return result[1]

In [68]:
locations_stipend = data[['location','stipend']]

grouping_locations_with_occurences=2
grouping_locations_with_occurences-=1 #To overcome '+1' in '*' line
while True:
    locations_stipend = data[['location','stipend']] #To overcome slicing
    res = perform_anova_for_locations(locations_stipend,grouping_locations_with_occurences+1) # '*' line
    if res<0.05:
        break
    grouping_locations_with_occurences+=1

print("Locations with ",grouping_locations_with_occurences,' occurences can be grouped')

ANOVA result for grouping locations with  2  occurences : F_onewayResult(statistic=1.4496209438671848, pvalue=0.16258368763713887)
ANOVA result for grouping locations with  3  occurences : F_onewayResult(statistic=2.3224456637226734, pvalue=0.0027380500505397817)
Locations with  2  occurences can be grouped


C:\Users\avikm\AppData\Local\Temp\ipykernel_17132\3077030034.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  locations_stipend_df.loc[:,'least_location'] = (locations_stipend_df['location'].apply(lambda x: leastOccuringLocation(str(x).split(',')[:-1],numberOfLocationsToBeGrouped)))
C:\Users\avikm\AppData\Local\Temp\ipykernel_17132\3077030034.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  locations_stipend_df.dropna(subset=['least_location'],inplace=True)
C:\Users\avikm\AppData\Local\Temp\ipykernel_17132\3077030034.py:15: SettingWithCopyWarning: 
A valu

In [69]:
locations_to_be_edited = reverse_location_dict.copy()
key_list = list(locations_to_be_edited.keys())
for key in key_list:
    if locations_to_be_edited.get(key)<=grouping_locations_with_occurences:
        locations_to_be_edited.update({key:'Others'})
    else:
        locations_to_be_edited.update({key:key})

unique_final_locations = (locations_to_be_edited.values())
print("Unique Locations after merging not significant locations as 'Others': ",pd.Series(unique_final_locations).nunique())

def modify_list(location_list):
    new_list = []
    for list_item in location_list:
        new_list.append(locations_to_be_edited.get(list_item))
    return new_list

data_locations = pd.DataFrame(data['location'].apply(lambda x: modify_list(str(x).split(',')[:-1])))


Unique Locations after merging not significant locations as 'Others':  87


In [70]:
unique_significant_locations = list(pd.Series(locations_to_be_edited.values()).unique())
loc = []
stip = []
for index, row in data.iterrows():
    locations = (row['location']).split(',')[:-1]
    for location in locations:
        if location in unique_significant_locations:
            loc.append(location)
        else:
            loc.append('Others')
        stip.append(row['stipend'])
location_stipend_data = pd.DataFrame(list(zip(loc,stip)),columns = ['location','stipend'])

In [71]:
perk_array = data['perks'].apply(lambda x: str(x).split(',')[:-1])
# Creating a 1d list
perk_list = [item for sublist in perk_array for item in sublist]

#Counting the occurences of each skill
from collections import Counter
perk_list = dict(Counter(perk_list))

# Sorting in ascending order for later use
sorted_perk_list = sorted(perk_list.keys())
# Sorting in descending order for plotting purpose
perks_list_toPlot = dict(sorted(perk_list.items(),key=lambda x:x[1], reverse=True))

print("All Location with their occurences: ",perks_list_toPlot)
print("Total unique perks: ",len(perks_list_toPlot))
data_perks = pd.DataFrame(data['perks'].apply(lambda x: (str(x).split(',')[:-1])))

All Location with their occurences:  {'Certificate': 3889, 'Letter of recommendation': 3060, 'Flexible work hours': 2051, '5 days a week': 1549, 'Informal dress code': 1300, 'Job offer': 1046, 'Free snacks & beverages': 569}
Total unique perks:  7


In [72]:
from sklearn.preprocessing import MultiLabelBinarizer
mlb = MultiLabelBinarizer()

sklist = list(data_skills['skills_required'])
skills_dummy_values = mlb.fit_transform(sklist)
_ = mlb.classes_
skills_dummy_classes = ["skill_"+x for x in _]
skills_dummies = pd.DataFrame(skills_dummy_values,columns=skills_dummy_classes, index=data.index)
skills_dummies.sample(3)

,skill_.NET,skill_3ds Max,skill_AJAX,skill_ANSYS,skill_ARM Microcontroller,skill_ASP.NET,skill_Accounting,skill_Acting Audition,skill_Acting technique,skill_Adobe After Effects,...,skill_Vue Js,skill_Webflow,skill_Wireframing,skill_WordPress,skill_XML,skill_Xcode,skill_YouTube Ads,skill_iOS,skill_jQuery,skill_web3.js
2983,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
316,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
378,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [73]:
from sklearn.preprocessing import MultiLabelBinarizer
mlb = MultiLabelBinarizer()

loclist = list(data_locations['location'])
location_dummy_values = mlb.fit_transform(loclist)
_ = mlb.classes_
location_dummy_classes = ["location_"+x for x in _]
location_dummies = pd.DataFrame(location_dummy_values,columns=location_dummy_classes, index=data.index)
location_dummies.sample(3)

,location_Agra,location_Ahmedabad,location_Ajmer,location_Aligarh,location_Ambernath,location_Amritsar,location_Aurangabad,location_Badarpur,location_Badlapur,location_Bangalore,...,location_Vadodara,location_Varanasi,location_Vasai,location_Vasai-Virar,location_Vashi,location_Vijayawada,location_Visakhapatnam,location_Warangal,location_Work From Home,location_Zirakpur
9077,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
6420,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
2115,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [74]:
from sklearn.preprocessing import MultiLabelBinarizer
mlb = MultiLabelBinarizer()

perklist = list(data_perks['perks'])
perk_dummy_values = mlb.fit_transform(perklist)
_ = mlb.classes_
perk_dummy_classes = ["perk_"+x for x in _]
perk_dummies = pd.DataFrame(perk_dummy_values,columns=perk_dummy_classes, index=data.index)
perk_dummies.sample(3)

,perk_5 days a week,perk_Certificate,perk_Flexible work hours,perk_Free snacks & beverages,perk_Informal dress code,perk_Job offer,perk_Letter of recommendation
5453,1,0,0,1,1,0,1
8142,0,1,1,0,0,0,0
4748,0,1,1,0,0,0,0


In [75]:
category_dummies = pd.get_dummies(data[['category']])

In [76]:
import re

# Function to remove special characters from feature names
def clean_feature_names(feature_names):
    cleaned_names = []
    for name in feature_names:
        cleaned_name = re.sub(r'[^\w\s]', '', name)  # Remove special characters
        cleaned_names.append(cleaned_name)
    return cleaned_names

In [77]:
from sklearn.discriminant_analysis import StandardScaler
# Import necessary libraries
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
# Combine all the preprocessed data into one DataFrame
X = pd.concat([skills_dummies, location_dummies, perk_dummies, data['duration'], category_dummies, data['stipend']], axis=1)


In [78]:

# Define the target variable
y = data['company']


In [79]:
cols = X.dtypes[X.dtypes==bool].index.to_list()
for col in cols:
    X[col] = X[col].astype(int)

In [80]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
y_t = label_encoder.fit_transform(y)

In [81]:
import xgboost as xgb
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

# Scale features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
# Apply SMOTE to balance the classes in the entire dataset
smote = SMOTE(k_neighbors=1,random_state=42)
X_resampled, y_resampled = smote.fit_resample(X_scaled, y_t)


In [82]:
import pickle

with open("X_scaled.pkl","wb") as f:
    pickle.dump(X_scaled, f)
    

with open("y_t.pkl","wb") as f:
    pickle.dump(y_t, f)    


In [83]:

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.3, random_state=42)


In [84]:
print("Fitting model RC")
clf = RandomForestClassifier(max_depth=100, random_state=42, verbose=2)
clf.fit(X_train,y_train)
y_train_pred = clf.predict(X_train)


# Predict on the test set
y_test_pred = clf.predict(X_test)

# Calculate accuracy
accuracy1 = accuracy_score(y_train, y_train_pred)
accuracy2 = accuracy_score(y_test, y_test_pred)
clfrepor1 = classification_report(y_train, y_train_pred)
clfrepor2 = classification_report(y_test, y_test_pred)
print("Accuracy on train:", accuracy1)
print(clfrepor1)

print("Accuracy on test:", accuracy2)
print(clfrepor2)


Fitting model RC
building tree 1 of 100
building tree 2 of 100
building tree 3 of 100
building tree 4 of 100
building tree 5 of 100
building tree 6 of 100
building tree 7 of 100
building tree 8 of 100
building tree 9 of 100
building tree 10 of 100
building tree 11 of 100
building tree 12 of 100
building tree 13 of 100
building tree 14 of 100
building tree 15 of 100
building tree 16 of 100
building tree 17 of 100
building tree 18 of 100
building tree 19 of 100
building tree 20 of 100
building tree 21 of 100
building tree 22 of 100
building tree 23 of 100
building tree 24 of 100
building tree 25 of 100
building tree 26 of 100
building tree 27 of 100
building tree 28 of 100
building tree 29 of 100
building tree 30 of 100
building tree 31 of 100
building tree 32 of 100
building tree 33 of 100
building tree 34 of 100
building tree 35 of 100
building tree 36 of 100
building tree 37 of 100
building tree 38 of 100
building tree 39 of 100
building tree 40 of 100


[Parallel(n_jobs=1)]: Done  40 tasks      | elapsed:  2.8min


building tree 41 of 100
building tree 42 of 100
building tree 43 of 100
building tree 44 of 100
building tree 45 of 100
building tree 46 of 100
building tree 47 of 100
building tree 48 of 100
building tree 49 of 100
building tree 50 of 100
building tree 51 of 100
building tree 52 of 100
building tree 53 of 100
building tree 54 of 100
building tree 55 of 100
building tree 56 of 100
building tree 57 of 100
building tree 58 of 100
building tree 59 of 100
building tree 60 of 100
building tree 61 of 100
building tree 62 of 100
building tree 63 of 100
building tree 64 of 100
building tree 65 of 100
building tree 66 of 100
building tree 67 of 100
building tree 68 of 100
building tree 69 of 100
building tree 70 of 100
building tree 71 of 100
building tree 72 of 100
building tree 73 of 100
building tree 74 of 100
building tree 75 of 100
building tree 76 of 100
building tree 77 of 100
building tree 78 of 100
building tree 79 of 100
building tree 80 of 100
building tree 81 of 100
building tree 82

[Parallel(n_jobs=1)]: Done  40 tasks      | elapsed:   52.6s
[Parallel(n_jobs=1)]: Done  40 tasks      | elapsed:   15.7s
c:\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _war

Accuracy on train: 0.8918979588564465
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        44
           1       1.00      1.00      1.00        45
           2       1.00      1.00      1.00        37
           3       1.00      1.00      1.00        36
           4       1.00      1.00      1.00        42
           5       1.00      1.00      1.00        44
           6       1.00      0.97      0.99        40
           7       1.00      1.00      1.00        39
           8       1.00      1.00      1.00        40
           9       1.00      0.98      0.99        46
          10       1.00      1.00      1.00        38
          11       1.00      1.00      1.00        36
          12       1.00      1.00      1.00        41
          13       0.98      1.00      0.99        41
          14       0.00      0.00      0.00        36
          15       1.00      0.98      0.99        45
          16       1.00      0.97      0.99

c:\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:

# Initialize and train the XGBoost classifier
print("Fitting model")
clf = xgb.XGBClassifier(max_depth=100, random_state=42, verbosity=2)
clf.fit(X_train, y_train)

# Predict on the train set
y_train_pred = clf.predict(X_train)

# Predict on the test set
y_test_pred = clf.predict(X_test)

# Calculate accuracy and classification report
accuracy1 = accuracy_score(y_train, y_train_pred)
accuracy2 = accuracy_score(y_test, y_test_pred)
clfrepor1 = classification_report(y_train, y_train_pred)
clfrepor2 = classification_report(y_test, y_test_pred)

print("Accuracy on train:", accuracy1)
print(clfrepor1)

print("Accuracy on test:", accuracy2)
print(clfrepor2)


In [85]:
import pickle

with open("rc_clf.pkl","wb") as f:
    pickle.dump(clf, f)


In [63]:
pd.set_option("display.max_rows", 20)

In [56]:
X['category_Management'].sum()

2

In [ ]:
pd.DataFrame(X.sum().sort_values(ascending=False))